<a href="https://colab.research.google.com/github/AlexOchoki/Investigate-a-Dataset-TMDb-Movies-Dataset/blob/main/Improved_Aug_MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
# 📌 At the very top of the notebook
import time
notebook_start_time = time.time()


In [144]:
# Install (if needed) and import libraries for Excel file handling
!pip install --quiet openpyxl xlrd

# Core libraries
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path
# Excel workbook operations (optional: only if you modify Excel structure like formatting, formulas)
from openpyxl import load_workbook


In [145]:
# Define base folders
base_dir = Path('/content/drive/MyDrive/SNS/Ethiopia/2025/aug/dirty')
clean_dir = Path('/content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean')
constants_dir = Path('/content/drive/MyDrive/SNS/Ethiopia/eth_constants')

# File paths
dirty_file = base_dir / 'aug_dirty.xlsx'
previous_month_file = base_dir / 'clean_maxerience_jul.xlsx'
ired_file = base_dir / 'iRED_MD_jul.xlsx'
latest_coordinates_file = constants_dir / 'coordinates.xlsx'

# Mapping file and its sheets
mapping_file = constants_dir / 'eth_mappings.xlsx'

In [146]:
# Load main data
dirty_df = pd.read_excel(dirty_file)
previous_month_df = pd.read_excel(previous_month_file)
ired_df = pd.read_excel(ired_file)
latest_coordinates_df = pd.read_excel(latest_coordinates_file)

# Load mapping sheets
sheet_names = {
    'picos_regions': 'picos_regions',
    'subchannel_map': 'subchannel_to_trade_channel',
    'users': 'new_user_credentials',
    'rsm': 'rsm_map',
    'asm': 'asm_map',
    'csr': 'csr_map',
    'week_dates': 'week_dates_map'
}

picos_regions_df = pd.read_excel(mapping_file, sheet_name=sheet_names['picos_regions'])
subchannel_map_df = pd.read_excel(mapping_file, sheet_name=sheet_names['subchannel_map'])
users_df = pd.read_excel(mapping_file, sheet_name=sheet_names['users'])
rsm_df = pd.read_excel(mapping_file, sheet_name=sheet_names['rsm'])
asm_df = pd.read_excel(mapping_file, sheet_name=sheet_names['asm'])
csr_df = pd.read_excel(mapping_file, sheet_name=sheet_names['csr'])
week_dates_df = pd.read_excel(mapping_file, sheet_name=sheet_names['week_dates'])


In [147]:
# Define base folders
base_dir = Path('/content/drive/MyDrive/SNS/Ethiopia/2025/aug/dirty')
constants_dir = Path('/content/drive/MyDrive/SNS/Ethiopia/eth_constants')

# Define file paths
dirty_file = base_dir / 'aug_dirty.xlsx'
previous_month_file = base_dir / 'clean_maxerience_jul.xlsx'
ired_file = base_dir / 'iRED_MD_jul.xlsx'
latest_coordinates_file = constants_dir / 'coordinates.xlsx'
mapping_file = constants_dir / 'eth_mappings.xlsx'

# Load main data
dirty_df = pd.read_excel(dirty_file)
previous_month_df = pd.read_excel(previous_month_file)
ired_df = pd.read_excel(ired_file)
latest_coordinates_df = pd.read_excel(latest_coordinates_file)

# Load mapping sheets (your clean version)
picos_regions_df = pd.read_excel(mapping_file, sheet_name='picos_regions')
subchannel_map_df = pd.read_excel(mapping_file, sheet_name='subchannel_to_trade_channel')
users_df = pd.read_excel(mapping_file, sheet_name='new_user_credentials')
rsm_df = pd.read_excel(mapping_file, sheet_name='rsm_map')
asm_df = pd.read_excel(mapping_file, sheet_name='asm_map')
csr_df = pd.read_excel(mapping_file, sheet_name='csr_map')
week_dates_df = pd.read_excel(mapping_file, sheet_name='week_dates_map')


In [148]:
# Set pandas display options to show all columns and prevent column truncation
pd.set_option('display.max_columns', None)      # Show all columns
pd.set_option('display.max_rows', 100)          # Optional: control max rows in interactive output
pd.set_option('display.max_colwidth', None)     # Show full column content without truncation
# pd.set_option('display.width', 1000)          # Uncomment if you want wide-format printing in terminals

# Preview first 5 rows of raw data
dirty_df.head(5)


,RSM Ref.,ASM Ref.,CSR Ref.,Outlet SAP,MSD Code,Sales Territory,Region,DISTRICT,City,RSM Name,ASM Name,OCCD Code,OCCD NAME,CSR Name,CSR SAP,Surv. No.,Week day,rout Status,Sub route Number,Call Day,EDS Code,Customer Name,Owner name,Land Mark,Alternative GPS,Y - Coordinat,X _ Coordinat,Phone number,Channel Code,Sub Channel,Main Channel,Sub Channel.1,Coke Cooler Availability,Cooler Type,Cooler Serial number,Cooler asset Number,KO Weekly RGB VPO,KO Weekly PET VPO,4.Ambo Weekly RGB VPO,Ambo Weekly PET VPO,RGB weekly VPO,PET weekly VPO,Total Weekly,RGB Monthly VPO,PET Monthly VPO,Monthly VPO,Leveling
0,RSM 1,ASM 27,CSR 50,7777912,11275504,Addis Ababa,SOUTH & WEST ADDIS,SOUTH ADDIS,Addis City,MOHAMMED AHMED,HANA ZERFU,7006341.0,LIDIYA GORFU,ABEL G/MICHAEL,74925,13,4.0,RED,14.0,2,TRR1305,Kebe restaurant,buruk,58,8.9354 38.747,8.9354,38.747,923792779,151 Fine restaurant,HORECA/CAFE,ON-PREMISES,HORECA/CAFE,Cooler,Cooler,ET00123744,No Asset,4,0,3.0,0.0,7.0,0.0,7.0,30.31,0.0,30.31,Bronze
1,RSM 1,ASM 27,CSR 50,7773991,11271759,Addis Ababa,SOUTH & WEST ADDIS,SOUTH ADDIS,Addis City,MOHAMMED AHMED,HANA ZERFU,7006341.0,LIDIYA GORFU,ABEL G/MICHAEL,74925,13,4.0,RED,14.0,2,TRR1502,Mame Grossery,Abreham kebru,58,8.9374 38.7435,8.9374,38.7435,912724119,200 Grocery (Bar),BUTCHERY,ON-PREMISES,BUTCHERY,None Cooler,None Cooler,None Cooler,None Cooler,4,0,1.0,0.0,5.0,0.0,5.0,21.65,0.0,21.65,Bronze
2,RSM 1,ASM 27,CSR 50,7773990,11271758,Addis Ababa,SOUTH & WEST ADDIS,SOUTH ADDIS,Addis City,MOHAMMED AHMED,HANA ZERFU,7006341.0,LIDIYA GORFU,ABEL G/MICHAEL,74925,13,4.0,RED,16.0,4,TRR1501,Kedus bar and restaurant,Teruwerk lengerew,hana kelebet,8.9367 38.7438,8.9367,38.7438,911666873,151 Fine restaurant,HORECA/CAFE,ON-PREMISES,HORECA/CAFE,None Cooler,None Cooler,None Cooler,None Cooler,2,0,0.0,0.0,2.0,0.0,2.0,8.66,0.0,8.66,Bronze
3,RSM 1,ASM 27,CSR 50,7773977,11271745,Addis Ababa,SOUTH & WEST ADDIS,SOUTH ADDIS,Addis City,MOHAMMED AHMED,HANA ZERFU,7006341.0,LIDIYA GORFU,ABEL G/MICHAEL,74925,13,4.0,RED,14.0,2,TRR1421,Yosef makefafeya,yosef birhanu,58,8.9371 38.7456,8.9371,38.7456,911807227,351 Beverage wholesaler,LIQUOR,LIQUOR (LIQUOR STORE),LIQUOR,None Cooler,None Cooler,None Cooler,None Cooler,4,0,1.0,0.0,5.0,0.0,5.0,21.65,0.0,21.65,Bronze
4,RSM 1,ASM 27,CSR 50,7773967,11271735,Addis Ababa,SOUTH & WEST ADDIS,SOUTH ADDIS,Addis City,MOHAMMED AHMED,HANA ZERFU,7006341.0,LIDIYA GORFU,ABEL G/MICHAEL,74925,13,4.0,RED,14.0,2,TRR1414,Dagim hotel,tewabch tamne,58,8.9389 38.7475,8.9389,38.7475,911359932,401 Hotel,HORECA/CAFE,ON-PREMISES,HORECA/CAFE,None Cooler,None Cooler,None Cooler,None Cooler,1,0,1.0,0.0,2.0,0.0,2.0,8.66,0.0,8.66,Bronze


In [149]:
print("===== dirty_df Exploration =====\n")

# 1. Shape of the DataFrame
print(f"Shape (Rows, Columns): {dirty_df.shape}")

# 2. Missing values: count and percentage
missing_count = dirty_df.isnull().sum().astype(int)
missing_pct = (dirty_df.isnull().mean() * 100).round(2)

# 3. Build summary DataFrame
missing_summary = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing %': missing_pct.map(lambda x: f"{int(x) if x.is_integer() else x}%")
})

# 4. Display summary
print("\nMissing Values Summary:")
display(missing_summary)

# 5. Save to clean folder
clean_dir.mkdir(parents=True, exist_ok=True)
output_path = clean_dir / 'missing_summary.xlsx'
missing_summary.to_excel(output_path, index=True)

print(f"\n✅ Saved missing value summary to: {output_path}")
print("\n" + "=" * 50)


===== dirty_df Exploration =====

Shape (Rows, Columns): (25537, 47)

Missing Values Summary:


,Missing Count,Missing %
RSM Ref.,0,0%
ASM Ref.,0,0%
CSR Ref.,0,0%
Outlet SAP,9547,37.38%
MSD Code,0,0%
Sales Territory,0,0%
Region,0,0%
DISTRICT,0,0%
City,297,1.16%
RSM Name,0,0%



✅ Saved missing value summary to: /content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/missing_summary.xlsx



In [150]:
# List of important columns to check
important_columns = [
    "RSM Ref.", "ASM Ref.", "CSR Ref.", "MSD Code", "RSM Name",
    "ASM Name", "CSR Name", "Surv. No.", "Week day", "Customer Name", "Leveling",
    "Sub Channel", "Coke Cooler Availability"
]

print("===== Key Column Null Check Summary =====\n")

total_rows = len(dirty_df)

for col in important_columns:
    null_count = dirty_df[col].isnull().sum()
    null_pct = (null_count / total_rows) * 100

    if null_count > 0:
        print(f"⚠️  '{col}': {null_count} nulls ({null_pct:.2f}%) — recheck or ask the Country Champion to reshare the file.")
    else:
        print(f"✅  '{col}' has no missing values.")

print("\n" + "=" * 50)

===== Key Column Null Check Summary =====

✅  'RSM Ref.' has no missing values.
✅  'ASM Ref.' has no missing values.
✅  'CSR Ref.' has no missing values.
✅  'MSD Code' has no missing values.
✅  'RSM Name' has no missing values.
✅  'ASM Name' has no missing values.
✅  'CSR Name' has no missing values.
✅  'Surv. No.' has no missing values.
✅  'Week day' has no missing values.
✅  'Customer Name' has no missing values.
✅  'Leveling' has no missing values.
⚠️  'Sub Channel': 36 nulls (0.14%) — recheck or ask the Country Champion to reshare the file.
✅  'Coke Cooler Availability' has no missing values.



In [151]:

# Define important columns to check for completeness
important_columns = [
    "RSM Ref.", "ASM Ref.", "CSR Ref.", "MSD Code", "RSM Name",
    "ASM Name", "CSR Name", "Surv. No.", "Week day", "Customer Name", "Leveling",
    "Sub Channel", "Coke Cooler Availability"
]

print("===== Key Column Null Check Summary =====\n")

rows_before = len(dirty_df)
nulls_removed = []

# Drop rows with nulls in any of the important columns, and track removals
for col in important_columns:
    null_count = dirty_df[col].isnull().sum()
    null_pct = (null_count / len(dirty_df)) * 100

    if null_count > 0:
        print(f"⚠️  '{col}': {null_count} nulls ({null_pct:.2f}%) — rows will be removed.")
        nulls_removed.append((col, null_count))
        dirty_df = dirty_df[dirty_df[col].notnull()]  # Drop rows with null in this column
    else:
        print(f"✅  '{col}' has no missing values.")

rows_after = len(dirty_df)
rows_dropped = rows_before - rows_after

# Summary
print(f"\n🧹 Total rows before cleaning: {rows_before}")
print(f"🧹 Total rows after cleaning:  {rows_after}")
print(f"🗑️  Total rows removed:        {rows_dropped}\n")

for col, count in nulls_removed:
    print(f"   • {count} rows removed due to nulls in '{col}'")

print("\n" + "=" * 50)



===== Key Column Null Check Summary =====

✅  'RSM Ref.' has no missing values.
✅  'ASM Ref.' has no missing values.
✅  'CSR Ref.' has no missing values.
✅  'MSD Code' has no missing values.
✅  'RSM Name' has no missing values.
✅  'ASM Name' has no missing values.
✅  'CSR Name' has no missing values.
✅  'Surv. No.' has no missing values.
✅  'Week day' has no missing values.
✅  'Customer Name' has no missing values.
✅  'Leveling' has no missing values.
⚠️  'Sub Channel': 36 nulls (0.14%) — rows will be removed.
✅  'Coke Cooler Availability' has no missing values.

🧹 Total rows before cleaning: 25537
🧹 Total rows after cleaning:  25501
🗑️  Total rows removed:        36

   • 36 rows removed due to nulls in 'Sub Channel'



### 🧹 Step 1: Strip Whitespace and Clean Strings

In [152]:
# Strip leading/trailing spaces and reduce multiple internal spaces to one
for col in dirty_df.columns:
    dirty_df[col] = dirty_df[col].apply(
        lambda x: re.sub(r'\s+', ' ', str(x).strip()) if pd.notnull(x) else x
    )

### 🧮 Step: Clean and Validate 'MSD Code' Column

This step ensures that:
- All `MSD Code` values are cleaned and converted to whole numbers (integers)
- Any non-numeric or invalid entries are flagged for correction
- Duplicate MSD Codes are identified and summarized

This is critical for maintaining master data integrity, especially where `MSD Code` acts as a unique or semi-unique key.


In [153]:
# === 🧼 Clean & Validate 'MSD Code' Column ===

if 'MSD Code' in dirty_df.columns:
    # 1. Convert to numeric, invalids to NaN
    dirty_df['MSD Code'] = pd.to_numeric(dirty_df['MSD Code'], errors='coerce')

    # 2. Identify invalid (non-numeric) or decimal entries
    invalid_entries = dirty_df[dirty_df['MSD Code'].isna() | (dirty_df['MSD Code'] % 1 != 0)]
    num_invalid = len(invalid_entries)

    if num_invalid > 0:
        print(f"⚠️ {num_invalid} entries in 'MSD Code' are either non-numeric or have decimal places.")
    else:
        print("✅ All 'MSD Code' values are valid whole numbers.")

    # 3. Drop invalid or decimal MSD Codes (optional)
    dirty_df = dirty_df[dirty_df['MSD Code'].notna() & (dirty_df['MSD Code'] % 1 == 0)]

    # 4. Convert to nullable integer type
    dirty_df['MSD Code'] = dirty_df['MSD Code'].astype('Int64')

    # 5. Check for duplicates
    duplicate_counts = dirty_df['MSD Code'].value_counts()
    duplicates = duplicate_counts[duplicate_counts > 1]
    total_duplicates = duplicates.sum()

    print("\n===== MSD Code Summary =====")
    print(f"🔍 Unique duplicated MSD Codes: {len(duplicates)}")
    print(f"📌 Total duplicate rows: {total_duplicates}")

    if not duplicates.empty:
        print("\nTop 5 Duplicate MSD Codes:")
        print(duplicates.head())

    print("\n===== Final Summary =====")
    if total_duplicates > 0:
        print(f"⚠️ {total_duplicates} duplicate rows found — please verify.")
    else:
        print("✅ No duplicate MSD Codes found.")
else:
    print("❌ 'MSD Code' column not found in the DataFrame.")


✅ All 'MSD Code' values are valid whole numbers.

===== MSD Code Summary =====
🔍 Unique duplicated MSD Codes: 93
📌 Total duplicate rows: 186

Top 5 Duplicate MSD Codes:
MSD Code
11145713    2
11138332    2
11138326    2
11138323    2
11138322    2
Name: count, dtype: Int64

===== Final Summary =====
⚠️ 186 duplicate rows found — please verify.


### 🧹 Step: Remove Invalid and Duplicate MSD Codes

This step performs the following cleanup operations:
1. Removes rows where `MSD Code` is missing or invalid (non-numeric or blank)
2. Deduplicates the `MSD Code` column, keeping only the first occurrence
3. Provides a summary of rows removed and final row count after cleanup


In [154]:
# === 🧼 Remove Invalid and Duplicate MSD Codes ===

# Step 1: Remove rows with missing or invalid MSD Codes
initial_rows = len(dirty_df)
dirty_df = dirty_df[dirty_df['MSD Code'].notna()]
after_invalid_removal = len(dirty_df)
invalid_removed = initial_rows - after_invalid_removal

print(f"✅ Removed {invalid_removed} rows with invalid or non-numeric MSD Codes.")

# Step 2: Remove duplicate MSD Codes, keep first occurrence
before_dedup = len(dirty_df)
dirty_df = dirty_df.drop_duplicates(subset=['MSD Code'], keep='first')
after_dedup = len(dirty_df)
duplicates_removed = before_dedup - after_dedup

print(f"✅ Removed {duplicates_removed} duplicate MSD Code entries.")

# Final Summary
print("\n===== MSD Code Cleaning Summary =====")
print(f"🧾 Initial rows:               {initial_rows}")
print(f"🗑️  Removed invalid MSD Codes: {invalid_removed}")
print(f"♻️  Removed duplicate MSDs:    {duplicates_removed}")
print(f"📊 Final cleaned rows:         {after_dedup}")

✅ Removed 0 rows with invalid or non-numeric MSD Codes.
✅ Removed 93 duplicate MSD Code entries.

===== MSD Code Cleaning Summary =====
🧾 Initial rows:               25501
🗑️  Removed invalid MSD Codes: 0
♻️  Removed duplicate MSDs:    93
📊 Final cleaned rows:         25408


### 🔍 Step: Check for Nulls in Key Columns

This step ensures that all critical fields (IDs, names, codes, and survey info) are fully populated.  
Any rows with missing values in these fields should be flagged for correction or exclusion from reporting.


In [155]:
# === 🔍 Null Check for Important Columns ===

important_columns = [
    "RSM Ref.", "ASM Ref.", "CSR Ref.", "MSD Code", "RSM Name",
    "ASM Name", "CSR Name", "Surv. No.", "Week day", "Customer Name", "Leveling",
    "Sub Channel", "Coke Cooler Availability"
]

print("===== Key Column Null Check Summary =====\n")

total_rows = len(dirty_df)

for col in important_columns:
    null_count = dirty_df[col].isnull().sum()
    null_pct = (null_count / total_rows) * 100

    if null_count > 0:
        print(f"⚠️  '{col}': {null_count} nulls ({null_pct:.2f}%) — recheck or ask the Country Champion to reshare the file.")
    else:
        print(f"✅  '{col}' has no missing values.")


===== Key Column Null Check Summary =====

✅  'RSM Ref.' has no missing values.
✅  'ASM Ref.' has no missing values.
✅  'CSR Ref.' has no missing values.
✅  'MSD Code' has no missing values.
✅  'RSM Name' has no missing values.
✅  'ASM Name' has no missing values.
✅  'CSR Name' has no missing values.
✅  'Surv. No.' has no missing values.
✅  'Week day' has no missing values.
✅  'Customer Name' has no missing values.
✅  'Leveling' has no missing values.
✅  'Sub Channel' has no missing values.
✅  'Coke Cooler Availability' has no missing values.


### 🧼 Step: Drop Rows with Nulls in Key Columns

This step ensures data integrity by:
- Checking for missing values in critical columns
- Removing rows where any of these key columns are null
- Summarizing how many rows were removed and why


In [156]:
# === 🧼 Drop Rows with Nulls in Key Columns ===

important_columns = [
    "RSM Ref.", "ASM Ref.", "CSR Ref.", "MSD Code", "RSM Name",
    "ASM Name", "CSR Name", "Surv. No.", "Week day", "Customer Name", "Leveling"
]

initial_rows = len(dirty_df)
removal_summary = {}

# Loop through each column and drop rows with nulls, tracking what’s removed
for col in important_columns:
    null_rows = dirty_df[dirty_df[col].isnull()]
    null_count = len(null_rows)

    if null_count > 0:
        dirty_df = dirty_df[dirty_df[col].notnull()]
        removal_summary[col] = null_count

final_rows = len(dirty_df)
total_removed = initial_rows - final_rows

# Print removal summary
print("\n===== Removal Summary =====")
if total_removed > 0:
    for col, count in removal_summary.items():
        print(f"🗑️  Removed {count} rows due to nulls in '{col}'")
    print(f"\n🔻 Total rows removed: {total_removed}")
else:
    print("✅ No rows removed. All key columns had complete data.")

print("\n" + "=" * 50)



===== Removal Summary =====
✅ No rows removed. All key columns had complete data.



### 🔁 Step: Validate One-to-One Mapping Between RSM Ref. and RSM Name

This step ensures that each `RSM Ref.` maps to exactly **one unique** `RSM Name`.  
Multiple names per reference could indicate data entry errors or ID reuse.

The script:
- Extracts all unique (Ref, Name) pairs
- Groups by Ref to count unique names
- Flags and prints any RSM Ref. that maps to more than one name


In [157]:
# === 🔁 Validate One-to-One Mapping: RSM Ref. ↔ RSM Name ===

# Step 1: Get distinct combinations
rsm_df = dirty_df[['RSM Ref.', 'RSM Name']].drop_duplicates()

# Step 2: Count how many unique names each Ref maps to
ref_counts = rsm_df.groupby('RSM Ref.')['RSM Name'].nunique()

# Step 3: Find conflicts (RSM Ref. mapped to multiple names)
conflicting_refs = ref_counts[ref_counts > 1]

# Step 4: Print results
print("===== Conflicting RSM Ref. to Name Mappings =====\n")
if conflicting_refs.empty:
    print("✅ All RSM Ref. values map to a single RSM Name.")
else:
    print("⚠️ The following RSM Ref. values map to multiple RSM Names:")
    for ref in conflicting_refs.index:
        names = rsm_df[rsm_df['RSM Ref.'] == ref]['RSM Name'].unique()
        print(f"🔁 {ref}: {list(names)}")


===== Conflicting RSM Ref. to Name Mappings =====

✅ All RSM Ref. values map to a single RSM Name.


### 🧾 Step: Standardize RSM Names Using Mapping Sheet

After validating that each `RSM Ref.` maps to a single unique name, we now:
- Create a mapping dictionary from the reference sheet
- Clean up formatting in `RSM Ref.` to ensure consistent keys
- Apply the mapping to generate a standardized `RSM Name (Cleaned)` column
- Identify any `RSM Ref.` values that were not mapped, which could indicate errors or typos


In [158]:
# === 🧾 Standardize RSM Names Using Mapping ===

# Step 1: Prepare mapping from validated RSM reference DataFrame
rsm_mapping_dict = dict(
    zip(
        rsm_df['RSM Ref.'].astype(str).str.strip(),
        rsm_df['RSM Name'].astype(str).str.strip()
    )
)

# Step 2: Clean up RSM Ref in dirty_df for accurate matching
dirty_df['RSM Ref.'] = dirty_df['RSM Ref.'].astype(str).str.strip()

# Step 3: Apply the mapping to get cleaned RSM Names
dirty_df['RSM Name (Cleaned)'] = dirty_df['RSM Ref.'].map(rsm_mapping_dict)

# Step 4: Identify any unmapped RSM Refs
unmapped_refs = dirty_df[dirty_df['RSM Name (Cleaned)'].isna()]['RSM Ref.'].unique()

# Step 5: Report mapping result
print("\n===== RSM Mapping Validation =====")
if len(unmapped_refs) > 0:
    print("⚠️ Unmapped RSM Refs found (please check):")
    print(unmapped_refs)
else:
    print("✅ All RSM Refs successfully mapped to cleaned names.")



===== RSM Mapping Validation =====
✅ All RSM Refs successfully mapped to cleaned names.


### 🔍 Step: Find RSM Names in dirty_df That Don't Match the Clean Mapping

This step identifies RSM Names in the `dirty_df` that don't match any of the standardized names in the reference `rsm_df`.
It helps highlight misspellings, incorrect spacing, or formatting inconsistencies.


In [159]:
# Step 1: Standardize formatting in both DataFrames
dirty_df['RSM Name'] = dirty_df['RSM Name'].astype(str).str.strip().str.lower()
rsm_df['RSM Name'] = rsm_df['RSM Name'].astype(str).str.strip().str.lower()

# Step 2: Create list of valid RSM names from rsm_df
valid_rsm_names = rsm_df['RSM Name'].unique()

# Step 3: Find dirty_df RSM Names that are not in the clean reference
invalid_rsm_df = (
    dirty_df[['RSM Ref.', 'RSM Name']]
    .drop_duplicates()
    .loc[~dirty_df['RSM Name'].isin(valid_rsm_names)]
)

# Step 4: Output results
print("===== RSM Names in dirty_df NOT Found in Reference Mapping =====\n")
if invalid_rsm_df.empty:
    print("✅ All RSM Names in dirty_df are present in the validated mapping.")
else:
    print("⚠️ Found invalid or unexpected RSM Names in dirty_df:")
    display(invalid_rsm_df.reset_index(drop=True))


===== RSM Names in dirty_df NOT Found in Reference Mapping =====

✅ All RSM Names in dirty_df are present in the validated mapping.


In [160]:
# === 🔍 Validate RSM Ref ↔ RSM Name Pair ===

# Standardize both sides
dirty_df['RSM Ref.'] = dirty_df['RSM Ref.'].astype(str).str.strip()
dirty_df['RSM Name'] = dirty_df['RSM Name'].astype(str).str.strip().str.title()
rsm_df['RSM Ref.'] = rsm_df['RSM Ref.'].astype(str).str.strip()
rsm_df['RSM Name'] = rsm_df['RSM Name'].astype(str).str.strip().str.title()

# Create and compare sets
valid_rsm_pairs = set(tuple(x) for x in rsm_df[['RSM Ref.', 'RSM Name']].drop_duplicates().values)
actual_rsm_pairs = set(tuple(x) for x in dirty_df[['RSM Ref.', 'RSM Name']].drop_duplicates().values)
invalid_rsm_pairs = actual_rsm_pairs - valid_rsm_pairs

# Show results
rsm_issues_df = pd.DataFrame(list(invalid_rsm_pairs), columns=['RSM Ref.', 'RSM Name'])
print("===== Invalid RSM Ref ↔ RSM Name Pairs in dirty_df =====\n")
if rsm_issues_df.empty:
    print("✅ All RSM Ref/Name pairs match the reference.")
else:
    display(rsm_issues_df)


===== Invalid RSM Ref ↔ RSM Name Pairs in dirty_df =====

✅ All RSM Ref/Name pairs match the reference.


### 🔁 Step: Validate One-to-One Mapping Between ASM Ref. and ASM Name

This step checks the consistency of the ASM hierarchy in the master data by validating that:
- Each `ASM Name` maps to only one `ASM Ref.`
- Each `ASM Ref.` maps to only one `ASM Name`

Any inconsistencies may point to duplicate references, typos, or misassignments that need correction.


In [161]:
# === 🔁 Validate One-to-One Mapping: ASM Ref. ↔ ASM Name ===

# Step 1: Clean and deduplicate ASM pairs
asm_df = dirty_df[['ASM Ref.', 'ASM Name']].drop_duplicates()
asm_df['ASM Ref.'] = asm_df['ASM Ref.'].astype(str).str.strip()
asm_df['ASM Name'] = asm_df['ASM Name'].astype(str).str.strip().str.lower()

print("===== ASM Mapping Validation =====\n")

# PART A — ASM Name mapped to multiple ASM Ref values
asm_name_to_multiple_refs = asm_df.groupby('ASM Name')['ASM Ref.'].nunique()
conflicting_names = asm_name_to_multiple_refs[asm_name_to_multiple_refs > 1]

print("❌ ASM Names mapped to more than one ASM Ref:")
if conflicting_names.empty:
    print("✅ All ASM Names map to a single ASM Ref.")
else:
    for name in conflicting_names.index:
        refs = asm_df[asm_df['ASM Name'] == name]['ASM Ref.'].unique()
        print(f"🔁 {name}: {list(refs)}")

# PART B — ASM Ref mapped to multiple ASM Name values
asm_ref_to_multiple_names = asm_df.groupby('ASM Ref.')['ASM Name'].nunique()
conflicting_refs = asm_ref_to_multiple_names[asm_ref_to_multiple_names > 1]

print("\n❌ ASM Refs mapped to more than one ASM Name:")
if conflicting_refs.empty:
    print("✅ All ASM Refs map to a single ASM Name.")
else:
    for ref in conflicting_refs.index:
        names = asm_df[asm_df['ASM Ref.'] == ref]['ASM Name'].unique()
        print(f"🔁 {ref}: {list(names)}")


===== ASM Mapping Validation =====

❌ ASM Names mapped to more than one ASM Ref:
🔁 birhanu a.: ['ASM 30', 'Asm 30']

❌ ASM Refs mapped to more than one ASM Name:
✅ All ASM Refs map to a single ASM Name.


### 🔍 Step: Validate ASM Names in dirty_df Against Reference csr_df

This step compares `ASM Name` values from the `dirty_df` against the clean master list in `csr_df`.  
It flags any `ASM Name` values in `dirty_df` that do not appear in the reference list, helping detect invalid, misspelled, or inconsistent entries.


In [162]:
# === 🔍 Compare dirty_df ASM Names against validated asm_df ===

# Step 1: Standardize formatting
dirty_df['ASM Name'] = dirty_df['ASM Name'].astype(str).str.strip().str.title()
asm_df['ASM Name'] = asm_df['ASM Name'].astype(str).str.strip().str.title()

# Step 2: Get list of valid ASM Names
valid_asm_names = asm_df['ASM Name'].unique()

# Step 3: Find ASM Names in dirty_df not present in asm_df
invalid_asm_df = (
    dirty_df[['ASM Ref.', 'ASM Name']]
    .drop_duplicates()
    .loc[~dirty_df['ASM Name'].isin(valid_asm_names)]
)

# Step 4: Print results
print("===== ASM Names in dirty_df NOT Found in asm_df Mapping =====\n")
if invalid_asm_df.empty:
    print("✅ All ASM Names in dirty_df match the reference list.")
else:
    print("⚠️ Found invalid or unexpected ASM Names in dirty_df:")
    display(invalid_asm_df.reset_index(drop=True))


===== ASM Names in dirty_df NOT Found in asm_df Mapping =====

✅ All ASM Names in dirty_df match the reference list.


In [163]:
# === 🔍 Validate ASM Ref ↔ ASM Name Pair ===

dirty_df['ASM Ref.'] = dirty_df['ASM Ref.'].astype(str).str.strip()
dirty_df['ASM Name'] = dirty_df['ASM Name'].astype(str).str.strip().str.title()
asm_df['ASM Ref.'] = asm_df['ASM Ref.'].astype(str).str.strip()
asm_df['ASM Name'] = asm_df['ASM Name'].astype(str).str.strip().str.title()

valid_asm_pairs = set(tuple(x) for x in asm_df[['ASM Ref.', 'ASM Name']].drop_duplicates().values)
actual_asm_pairs = set(tuple(x) for x in dirty_df[['ASM Ref.', 'ASM Name']].drop_duplicates().values)
invalid_asm_pairs = actual_asm_pairs - valid_asm_pairs

asm_issues_df = pd.DataFrame(list(invalid_asm_pairs), columns=['ASM Ref.', 'ASM Name'])
print("===== Invalid ASM Ref ↔ ASM Name Pairs in dirty_df =====\n")
if asm_issues_df.empty:
    print("✅ All ASM Ref/Name pairs match the reference.")
else:
    display(asm_issues_df)


===== Invalid ASM Ref ↔ ASM Name Pairs in dirty_df =====

✅ All ASM Ref/Name pairs match the reference.


### 🔁 Step: Validate One-to-One Mapping Between CSR Ref. and CSR Name

This step checks for inconsistencies in the CSR master data by ensuring:
- Each `CSR Name` maps to exactly one `CSR Ref.`
- Each `CSR Ref.` maps to exactly one `CSR Name`

Inconsistencies may indicate data entry errors, duplicated assignments, or naming mismatches.


In [164]:
# === 🔁 Validate One-to-One Mapping: CSR Ref. ↔ CSR Name ===

# Step 1: Clean and deduplicate CSR pairs
csr_df = dirty_df[['CSR Ref.', 'CSR Name']].drop_duplicates()
csr_df['CSR Ref.'] = csr_df['CSR Ref.'].astype(str).str.strip()
csr_df['CSR Name'] = csr_df['CSR Name'].astype(str).str.strip().str.title()

print("===== CSR Mapping Validation =====\n")

# PART A — CSR Name mapped to multiple CSR Ref values
csr_name_to_multiple_refs = csr_df.groupby('CSR Name')['CSR Ref.'].nunique()
conflicting_names = csr_name_to_multiple_refs[csr_name_to_multiple_refs > 1]

print("❌ CSR Names mapped to more than one CSR Ref:")
if conflicting_names.empty:
    print("✅ All CSR Names map to a single CSR Ref.")
else:
    for name in conflicting_names.index:
        refs = csr_df[csr_df['CSR Name'] == name]['CSR Ref.'].unique()
        print(f"🔁 {name}: {list(refs)}")

# PART B — CSR Ref mapped to multiple CSR Name values
csr_ref_to_multiple_names = csr_df.groupby('CSR Ref.')['CSR Name'].nunique()
conflicting_refs = csr_ref_to_multiple_names[csr_ref_to_multiple_names > 1]

print("\n❌ CSR Refs mapped to more than one CSR Name:")
if conflicting_refs.empty:
    print("✅ All CSR Refs map to a single CSR Name.")
else:
    for ref in conflicting_refs.index:
        names = csr_df[csr_df['CSR Ref.'] == ref]['CSR Name'].unique()
        print(f"🔁 {ref}: {list(names)}")


===== CSR Mapping Validation =====

❌ CSR Names mapped to more than one CSR Ref:
🔁 Mikiyas Belay: ['CSR 97', 'CSR 118']
🔁 Tewodross Nigus: ['CSR 67', 'CSR 39']

❌ CSR Refs mapped to more than one CSR Name:
🔁 CSR 12: ['Amare Gete', 'Tigist Tzegaw']
🔁 CSR 178: ['Adeladay M', 'Addisu Mulualem', 'Addisu Mulualem Ambaw']
🔁 CSR 55: ['Ebisa Hunde', 'Ephrem Dubale']


### 🔍 Step: Validate CSR Names in dirty_df Against Reference Mapping

This step compares all `CSR Name` values from the master data (`dirty_df`) against the cleaned and validated `CSR Name` list from `csr_df`.  
Any mismatched entries are flagged for correction (e.g. typos or invalid entries).


In [165]:
# === 🔍 Compare dirty_df CSR Names against validated csr_df ===

# Step 1: Standardize formatting in both dataframes
dirty_df['CSR Name'] = dirty_df['CSR Name'].astype(str).str.strip().str.title()
csr_df['CSR Name'] = csr_df['CSR Name'].astype(str).str.strip().str.title()

# Step 2: Extract the valid CSR names
valid_csr_names = csr_df['CSR Name'].unique()

# Step 3: Identify CSR Names in dirty_df that are not in the validated list
invalid_csr_df = (
    dirty_df[['CSR Ref.', 'CSR Name']]
    .drop_duplicates()
    .loc[~dirty_df['CSR Name'].isin(valid_csr_names)]
)

# Step 4: Output mismatches
print("===== CSR Names in dirty_df NOT Found in csr_df Mapping =====\n")
if invalid_csr_df.empty:
    print("✅ All CSR Names in dirty_df match the reference list.")
else:
    print("⚠️ Found invalid or unexpected CSR Names in dirty_df:")
    display(invalid_csr_df.reset_index(drop=True))


===== CSR Names in dirty_df NOT Found in csr_df Mapping =====

✅ All CSR Names in dirty_df match the reference list.


In [166]:
# === 🔍 Compare dirty_df CSR Names against validated csr_df ===

# Step 1: Standardize formatting in both dataframes
dirty_df['CSR Name'] = dirty_df['CSR Name'].astype(str).str.strip().str.title()
csr_df['CSR Name'] = csr_df['CSR Name'].astype(str).str.strip().str.title()

# Step 2: Extract the valid CSR names
valid_csr_names = csr_df['CSR Name'].unique()

# Step 3: Identify CSR Names in dirty_df that are not in the validated list
invalid_csr_df = (
    dirty_df[['CSR Ref.', 'CSR Name']]
    .drop_duplicates()
    .loc[~dirty_df['CSR Name'].isin(valid_csr_names)]
)

# Step 4: Output mismatches
print("===== CSR Names in dirty_df NOT Found in csr_df Mapping =====\n")
if invalid_csr_df.empty:
    print("✅ All CSR Names in dirty_df match the reference list.")
else:
    print("⚠️ Found invalid or unexpected CSR Names in dirty_df:")
    display(invalid_csr_df.reset_index(drop=True))


===== CSR Names in dirty_df NOT Found in csr_df Mapping =====

✅ All CSR Names in dirty_df match the reference list.


### 🌍 Step: Validate Region Values in dirty_df Against Previous Month

This step ensures that all `Region` values in the current dataset (`dirty_df`) exist in the validated list of `RegionName` values from the previous month.  
It helps catch typos, renamed regions, or unexpected new entries.


In [167]:
# === 🌍 Compare Region Values from dirty_df vs previous_month_df ===

# Step 1: Clean and extract unique region values from both datasets
regions_dirty_df = dirty_df[['Region']].drop_duplicates()
regions_dirty_df['Region'] = regions_dirty_df['Region'].astype(str).str.strip().str.title()

regions_prev_month = previous_month_df[['RegionName']].drop_duplicates()
regions_prev_month['RegionName'] = regions_prev_month['RegionName'].astype(str).str.strip().str.title()

# Step 2: Convert to sets for efficient comparison
regions_dirty = set(regions_dirty_df['Region'])
regions_prev = set(regions_prev_month['RegionName'])

# Step 3: Identify unmatched regions
non_matching_regions = sorted(list(regions_dirty - regions_prev))

# Step 4: Print results
print("===== Region Comparison =====\n")
if non_matching_regions:
    print("❌ Regions in 'dirty_df' NOT found in 'previous_month_df':")
    for region in non_matching_regions:
        print(f"- {region}")
else:
    print("✅ All regions in 'dirty_df' are present in 'previous_month_df'.")


===== Region Comparison =====

❌ Regions in 'dirty_df' NOT found in 'previous_month_df':
- Se
- South &West Addis


### 🔄 Region Mapping and Validation

This section helps clean mismatched `Region` values in `dirty_df` using a mapping based on discrepancies found when compared to the `previous_month_df`.


In [168]:
# === 🔍 Step 1: Extract and Normalize Unique Regions from Both DataFrames ===
regions_dirty_df = dirty_df[['Region']].drop_duplicates()
regions_dirty_df['Region'] = regions_dirty_df['Region'].astype(str).str.strip().str.title()

regions_prev_month = previous_month_df[['RegionName']].drop_duplicates()
regions_prev_month['RegionName'] = regions_prev_month['RegionName'].astype(str).str.strip().str.title()

# === 🔍 Step 2: Identify Regions in dirty_df Not Present in previous_month_df ===
regions_dirty = set(regions_dirty_df['Region'])
regions_prev = set(regions_prev_month['RegionName'])
non_matching_regions = sorted(list(regions_dirty - regions_prev))

# === 🧠 Step 3: Define Correction Map for Mismatched Regions ===
region_fix_map = {
    'Se': 'South Ethiopia',
    'South &West Addis': 'South & West Addis'
    # Add more fixes here if needed
}

# === 🛠️ Step 4: Apply the Mapping to dirty_df['Region'] ===
dirty_df['Region'] = dirty_df['Region'].astype(str).str.strip().str.title()
dirty_df['Region'] = dirty_df['Region'].replace(region_fix_map)

# === ✅ Step 5: Re-validate After Applying Fixes ===
updated_regions_dirty = set(dirty_df['Region'].unique())
non_matching_after_fix = sorted(list(updated_regions_dirty - regions_prev))

print("\n===== ✅ Region Mapping Verification =====")
if not non_matching_after_fix:
    print("🎉 All Region values in 'dirty_df' now match those in 'previous_month_df'.")
else:
    print("❌ The following Region values are still unmatched:")
    for region in non_matching_after_fix:
        print(f"- {region}")



===== ✅ Region Mapping Verification =====
🎉 All Region values in 'dirty_df' now match those in 'previous_month_df'.


### 🧭 Step: Validate CSR-to-ASM Hierarchy

This step verifies that each `CSR Name` is mapped to **only one `ASM Name`** in the hierarchy.  
If a CSR is linked to more than one ASM, it may indicate:
- A reassignment not reflected correctly in the data
- A miskeyed or inconsistent entry


In [169]:
# === 🧭 Check for CSR Reporting to Multiple ASM Names ===

# Step 1: Count how many unique ASM Names each CSR Name maps to
csr_asm_mapping = dirty_df.groupby('CSR Name')['ASM Name'].nunique()

# Step 2: Filter CSR Names with more than one ASM
non_compliant_csrs = csr_asm_mapping[csr_asm_mapping > 1]

# Step 3: Display results
print("===== CSR to ASM Hierarchy Check =====\n")
if not non_compliant_csrs.empty:
    print("❌ Non-compliant CSR entries found — CSR reporting to multiple ASM Names:\n")
    conflict_df = dirty_df[
        dirty_df['CSR Name'].isin(non_compliant_csrs.index)
    ][['CSR Ref.', 'CSR Name', 'ASM Ref.', 'ASM Name']].drop_duplicates().sort_values(by='CSR Name')

    display(conflict_df)

    # Optional: export to Excel
    # conflict_df.to_excel(clean_dir / 'csr_asm_conflicts.xlsx', index=False)
else:
    print("✅ Hierarchy is compliant — Each CSR reports to only one ASM.")


===== CSR to ASM Hierarchy Check =====

❌ Non-compliant CSR entries found — CSR reporting to multiple ASM Names:



,CSR Ref.,CSR Name,ASM Ref.,ASM Name
8375,CSR 97,Mikiyas Belay,ASM 4,Fitsum Girma
24151,CSR 118,Mikiyas Belay,ASM 9,Eskelealem Abebe
6329,CSR 67,Tewodross Nigus,ASM 3,Mohamed Eshetu
19733,CSR 39,Tewodross Nigus,ASM 12,Adiyam Tsegaye


### 🧭 Step: Validate ASM-to-RSM Hierarchy

This step verifies that each `ASM Name` is mapped to only **one `RSM Name`**.  
Any ASM reporting to multiple RSMs may indicate hierarchy conflicts due to:
- Team restructuring not fully updated
- Copy-paste errors or name inconsistencies


In [170]:
# === 🧭 Check for ASM Reporting to Multiple RSM Names ===

# Step 1: Count how many RSMs each ASM maps to
asm_rsm_mapping = dirty_df.groupby('ASM Name')['RSM Name'].nunique()

# Step 2: Identify ASMs mapped to more than one RSM
non_compliant_asms = asm_rsm_mapping[asm_rsm_mapping > 1]

# Step 3: Show violations if any
print("===== ASM to RSM Hierarchy Check =====\n")
if not non_compliant_asms.empty:
    print("❌ Non-compliant ASM entries found — ASM reporting to multiple RSM Names:\n")

    conflict_df = (
        dirty_df[dirty_df['ASM Name'].isin(non_compliant_asms.index)]
        [['ASM Ref.', 'ASM Name', 'RSM Ref.', 'RSM Name']]
        .drop_duplicates()
        .sort_values(by='ASM Name')
    )

    display(conflict_df)

    # Optional export
    # conflict_df.to_excel(clean_dir / 'asm_rsm_conflicts.xlsx', index=False)

else:
    print("✅ Hierarchy is compliant — Each ASM reports to only one RSM.")


===== ASM to RSM Hierarchy Check =====

✅ Hierarchy is compliant — Each ASM reports to only one RSM.


### 📏 Step: Check for Excessively Long Text in Key Columns

This function checks whether any values in specified columns exceed a given character limit (default: 100 characters).  
It flags each violation alongside the corresponding `MSD Code` for easy tracing.


In [171]:
def check_column_value_lengths(df, columns_to_check, max_length=100):
    """
    Checks whether values in specified columns exceed a given max character length.
    Prints any violations along with MSD Code and summary.

    Parameters:
    - df: pandas DataFrame (e.g., dirty_df)
    - columns_to_check: list of column names to validate
    - max_length: maximum allowed character length (default = 100)
    """
    for column in columns_to_check:
        # Filter rows where length exceeds max
        violations = df[df[column].astype(str).str.len() > max_length][['MSD Code', column]]

        print(f"\n===== Checking '{column}' for values > {max_length} characters =====")
        if not violations.empty:
            print(f"❌ Found {len(violations)} values exceeding limit in '{column}':")
            for _, row in violations.iterrows():
                print(f"MSD Code: {row['MSD Code']} — Value: {row[column]}")
        else:
            print(f"✅ All values in '{column}' are within the {max_length}-character limit.")

# 🟢 Columns to check
columns_to_validate = ['Customer Name', 'Land Mark']
check_column_value_lengths(dirty_df, columns_to_validate, max_length=100)



===== Checking 'Customer Name' for values > 100 characters =====
✅ All values in 'Customer Name' are within the 100-character limit.

===== Checking 'Land Mark' for values > 100 characters =====
✅ All values in 'Land Mark' are within the 100-character limit.


### 🧼 Step: Replace Invalid Entries and Remove Special Characters

This step:
- Replaces common unassigned values (e.g. 'XX', '', NaN) with `'Unassigned'`
- Removes special characters (e.g. `@!#$%^&*()` etc.) from cleaned text
- Outputs a summary of all replacements made


In [172]:


def replace_unassigned_and_clean(df, columns, unassigned_label='Unassigned'):
    """
    Replaces invalid/unassigned values and removes special characters in specified columns.

    Parameters:
    - df: pandas DataFrame to clean
    - columns: list of column names to clean
    - unassigned_label: value to replace invalid entries with (default = 'Unassigned')

    Returns:
    - cleaned DataFrame (modified in place)
    - dictionary of {column: replacement_count}
    """
    replace_values = ['XX', 'XXX', 'xx', 'xxx', '', 'nan']
    summary = {}

    for col in columns:
        if col not in df.columns:
            print(f"⚠️ Column '{col}' not found — skipping.")
            continue

        # Standardize and strip whitespace
        original = df[col].astype(str).str.strip()

        # Identify invalid values
        invalid_mask = original.isin(replace_values) | df[col].isna()
        count = invalid_mask.sum()

        # Replace invalids with 'Unassigned'
        df.loc[invalid_mask, col] = unassigned_label

        # Clean all remaining values — remove special characters but keep alphanumerics and spaces
        df[col] = df[col].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x).strip())

        summary[col] = count

    # Print summary
    print("===== Replacement & Cleaning Summary =====")
    for col, count in summary.items():
        print(f"✅ Replaced {count} values and cleaned special characters in '{col}'")

    return df, summary

# 🟢 Example usage
columns_to_clean = ['Customer Name', 'Land Mark', 'Sub route Number']
dirty_df, replacements = replace_unassigned_and_clean(dirty_df, columns_to_clean)


===== Replacement & Cleaning Summary =====
✅ Replaced 0 values and cleaned special characters in 'Customer Name'
✅ Replaced 2117 values and cleaned special characters in 'Land Mark'
✅ Replaced 7427 values and cleaned special characters in 'Sub route Number'


### ❄️ Step: Review Unique Values in 'Coke Cooler Availability'

This step extracts and displays all distinct, non-null values found in the `Coke Cooler Availability` column.  
This helps identify inconsistencies like extra spaces, unexpected categories, or mixed formats (e.g., 'Yes', 'yes ', 'NO').


In [173]:
# === ❄️ Review Unique 'Coke Cooler Availability' Values ===

# Step 1: Clean and get unique values
cooler_availability = (
    dirty_df['Coke Cooler Availability']
    .dropna()
    .astype(str)
    .str.strip()
    .str.title()  # Optional: Normalize case (e.g. 'yes', 'Yes' → 'Yes')
    .unique()
)

# Step 2: Sort for consistent viewing
cooler_availability = sorted(cooler_availability)

# Step 3: Display
print("✅ Unique values in 'Coke Cooler Availability':")
for val in cooler_availability:
    print(f"- {val}")


✅ Unique values in 'Coke Cooler Availability':
- Cooler
- No
- Non Cooler
- Non-Cooler
- Noncooler
- None
- None Cooler
- Yes


### 🧊 Step: Standardize Values in 'Coke Cooler Availability'

This step maps inconsistent labels like `'COOLER'` or `'Non cooler'` to their proper forms (`'Cooler'` and `'Non-Cooler'`).  
It ensures consistency for reliable filtering, grouping, and reporting.


In [174]:
# === 🧊 Normalize and Map 'Coke Cooler Availability' Values ===

# Step 1: Clean input formatting
dirty_df['Coke Cooler Availability'] = dirty_df['Coke Cooler Availability'].astype(str).str.strip()

# Step 2: Define grouped replacement mapping
replacement_groups = {
    'Cooler': ['Cooler', 'Yes'],
    'Non-Cooler': ['Non Cooler', 'Non-Cooler', 'Noncooler', 'None', 'None Cooler']
}

# Step 3: Build flat mapping dictionary
flat_map = {}
for clean_val, messy_vals in replacement_groups.items():
    for val in messy_vals:
        cleaned = re.sub(r'[^a-zA-Z\s]', '', val).strip().title()
        flat_map[cleaned] = clean_val

# Step 4: Count replacements
replacements_count = {}
for old_val, new_val in flat_map.items():
    count = (dirty_df['Coke Cooler Availability'].str.strip().str.title() == old_val).sum()
    replacements_count[(old_val, new_val)] = count

# Step 5: Apply the mapping
dirty_df['Coke Cooler Availability'] = (
    dirty_df['Coke Cooler Availability']
    .apply(lambda x: flat_map.get(re.sub(r'[^a-zA-Z\s]', '', x).strip().title(), x))
)

# Step 6: Print replacement summary
print("\n===== 'Coke Cooler Availability' Replacement Summary =====")
for (old_val, new_val), count in replacements_count.items():
    if count > 0:
        print(f"'{old_val}' → '{new_val}': {count} value(s) replaced")

print(f"\n✅ Total values replaced: {sum(replacements_count.values())}")



===== 'Coke Cooler Availability' Replacement Summary =====
'Cooler' → 'Cooler': 12297 value(s) replaced
'Yes' → 'Cooler': 75 value(s) replaced
'Non Cooler' → 'Non-Cooler': 599 value(s) replaced
'Noncooler' → 'Non-Cooler': 6 value(s) replaced
'None' → 'Non-Cooler': 4 value(s) replaced
'None Cooler' → 'Non-Cooler': 8077 value(s) replaced

✅ Total values replaced: 21058


### 🧮 Review Unique `Leveling` Values

This section extracts and displays the unique values present in the `Leveling` column after basic cleaning. The process:

- Strips leading/trailing whitespace  
- Converts to title case for consistency  
- Sorts values for easier reading  


In [175]:
# === 🧮 Review Unique 'Leveling' Values ===

# Step 1: Clean and get unique values
leveling_values = (
    dirty_df['Leveling']
    .dropna()
    .astype(str)
    .str.strip()
    .str.title()  # Normalize case for viewing (e.g. 'gold', 'Gold' → 'Gold')
    .unique()
)

# Step 2: Sort for consistent output
leveling_values = sorted(leveling_values)

# Step 3: Display
print("✅ Unique values in 'Leveling':")
for val in leveling_values:
    print(f"- {val}")


✅ Unique values in 'Leveling':
- Bronze
- Gold
- Silver


### 🧮 Step: Normalize and Standardize 'Leveling' Values

This step ensures that all entries in the `Leveling` column are consistently labeled as:
- `'Gold'`
- `'Silver'`
- `'Bronze'`

It handles inconsistencies like casing, partial spellings (e.g. `'bronz'`), and outputs a clean summary of the replacements made.


In [176]:
# === 🧊 Normalize and Map 'Leveling' Values ===

# Step 1: Clean input formatting (strip + lowercase)
dirty_df['Leveling'] = dirty_df['Leveling'].astype(str).str.strip().str.lower()

# Step 2: Define grouped mapping for messy variants
leveling_groups = {
    'Gold': ['gold', 'Gol'],
    'Silver': ['silver', 'Silvet'],
    'Bronze': ['bronze', 'bronz', 'Br9Nz', 'Broze']
}

# Step 3: Build flat mapping dictionary
flat_leveling_map = {}
for clean_val, messy_vals in leveling_groups.items():
    for val in messy_vals:
        cleaned = re.sub(r'[^a-zA-Z\s]', '', val).strip().lower()
        flat_leveling_map[cleaned] = clean_val

# Step 4: Count how many values match the raw keys before applying
replacements_count = {}
for raw_val, clean_val in flat_leveling_map.items():
    count = (dirty_df['Leveling'] == raw_val).sum()
    replacements_count[(raw_val, clean_val)] = count

# Step 5: Apply the flat mapping
dirty_df['Leveling'] = dirty_df['Leveling'].apply(
    lambda x: flat_leveling_map.get(re.sub(r'[^a-zA-Z\s]', '', x).strip().lower(), x)
)

# Step 6: Print replacement summary
print("\n===== 'Leveling' Replacement Summary =====")
for (old_val, new_val), count in replacements_count.items():
    if count > 0:
        print(f"'{old_val}' → '{new_val}': {count} value(s) replaced")

print(f"\n✅ Total values replaced: {sum(replacements_count.values())}")



===== 'Leveling' Replacement Summary =====
'gold' → 'Gold': 3022 value(s) replaced
'silver' → 'Silver': 8104 value(s) replaced
'bronze' → 'Bronze': 14282 value(s) replaced

✅ Total values replaced: 25408


### 🗺️ Map `Region` to `PICOS` Based on External Mapping

This section maps each region from the `dirty_df` to its corresponding `New PICOS` level using a reference DataFrame (`picos_regions_df`). The process includes:

- Stripping and normalizing the region names to lowercase  
- Dropping any nulls in the mapping reference  
- Creating a dictionary for quick lookup  
- Mapping `Region` in `dirty_df` to `New PICOS`  
- Checking for unmapped regions and printing a summary


In [177]:
# === 🗺️ Map 'Region' to 'PICOS' Using picos_regions_df ===

# Step 1: Create case-insensitive mapping dictionary
region_to_picos = (
    picos_regions_df
    .dropna(subset=['Region', 'New PICOS'])  # Drop invalid rows
    .set_index('Region')['New PICOS']
)

# Normalize keys in the mapping for consistent matching
region_to_picos.index = region_to_picos.index.str.strip().str.lower()
region_to_picos = region_to_picos.to_dict()

# Step 2: Map Region in dirty_df (normalized to lowercase)
dirty_df['PICOS_Region'] = (
    dirty_df['Region']
    .astype(str)
    .str.strip()
    .str.lower()
    .map(region_to_picos)
)

# Step 3: Identify and print any unmapped regions
unmapped_regions = dirty_df[dirty_df['PICOS_Region'].isna()]['Region'].unique()

print("===== PICOS Region Mapping Summary =====")
if len(unmapped_regions) > 0:
    print("⚠️ Regions not mapped to PICOS:")
    for region in unmapped_regions:
        print(f"- {region}")
else:
    print("✅ All regions successfully mapped to PICOS.")


===== PICOS Region Mapping Summary =====
✅ All regions successfully mapped to PICOS.


### 🔍 Extract & Export Unique Sub Channels

This section performs the following tasks:

- Cleans and extracts unique entries from the `Sub Channel` column in `dirty_df`
- Prepares a DataFrame for preview and validation
- Saves the list to an Excel file for manual review or mapping
- Ensures all text is stripped and standardized before export


In [178]:
# === 🔍 Extract & Export Unique Sub Channels ===

# Step 1: Normalize and extract unique subchannels
subchannels_df = (
    dirty_df['Sub Channel']
    .dropna()
    .astype(str)
    .str.strip()
    .str.title()
    .drop_duplicates()
    .to_frame(name='Unique Sub Channels')
)

# Step 2: Preview the first 10 unique subchannels
print("Preview of Unique Sub Channels:")
display(subchannels_df.head(10))

# Step 3: Export to Excel
output_file = "/content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/unique_subchannels.xlsx"
try:
    subchannels_df.to_excel(output_file, index=False)
    print(f"\n✅ Unique subchannels saved to: {output_file}")
except Exception as e:
    print(f"⚠️ Error saving the file: {e}")


Preview of Unique Sub Channels:


,Unique Sub Channels
0,Horeca/Cafe
1,Butchery
3,Liquor
6,Sook
11,E&D
41,Chat Bet
53,Mini Market
86,Juice Bet
113,Wholesaler
303,Super Market



✅ Unique subchannels saved to: /content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/unique_subchannels.xlsx


### 🔁 Normalize and Map 'Sub Channel' to 'Trade Channel'

This step standardizes the `Sub Channel` values in `dirty_df` using the reference mapping in `subchannel_map_df`.

**What this does:**
- Cleans the `Sub Channel` field (trims whitespaces).
- Maps it to standardized `Trade Channel` values using `subchannel_map_df`.
- Displays how many values were replaced.
- Highlights any values that were not mapped (still null).



## 🔁 Normalize and Case-Insensitive Map `'Sub Channel'` → `'Trade Channel'`

In [179]:

# Step 1: Normalize 'Sub Channel' column in dirty_df to lowercase and strip whitespace
dirty_df['Sub Channel'] = dirty_df['Sub Channel'].astype(str).str.strip()
dirty_df['_sub_lower'] = dirty_df['Sub Channel'].str.lower()

# Step 2: Normalize subchannel_map_df to lowercase for consistent mapping
mapping_df = subchannel_map_df.dropna(subset=['Sub Channel', 'Trade Channel']).copy()
mapping_df['Sub Channel'] = mapping_df['Sub Channel'].astype(str).str.strip().str.lower()
mapping_df['Trade Channel'] = mapping_df['Trade Channel'].astype(str).str.strip()

# Step 3: Build lowercase mapping dictionary
subchannel_map_dict = dict(zip(mapping_df['Sub Channel'], mapping_df['Trade Channel']))

# Step 4: Count how many values are present for each subchannel before mapping
original_counts = dirty_df['_sub_lower'].value_counts()

# Step 5: Apply mapping to the lowercase column and assign result to 'Trade Channel'
dirty_df['Trade Channel'] = dirty_df['_sub_lower'].map(subchannel_map_dict)

# Step 6: Create replacement summary
replacement_summary = {
    sub: original_counts[sub]
    for sub in subchannel_map_dict
    if sub in original_counts
}

# Step 7: Identify unmapped subchannels
unmapped_values = dirty_df[dirty_df['Trade Channel'].isna()]['Sub Channel'].unique()

# Step 8: Display results
print("\n📊 Sub Channel → Trade Channel Mapping Summary:")
for sub, count in replacement_summary.items():
    print(f"'{sub}' → '{subchannel_map_dict[sub]}': {count} value(s) replaced")

print(f"\n✅ Total replacements made: {sum(replacement_summary.values())}")

# Step 9: Display unmapped values
if len(unmapped_values) > 0:
    print("\n⚠️ Unmapped Sub Channel values:")
    for val in unmapped_values:
        print(f"- {val}")
else:
    print("\n✅ All Sub Channel values successfully mapped.")

# Optional: Drop helper column
dirty_df.drop(columns=['_sub_lower'], inplace=True)



📊 Sub Channel → Trade Channel Mapping Summary:
'104 sook' → 'L&T': 3 value(s) replaced
'351 beverage wholesaler' → 'Liquor Wholesale': 1 value(s) replaced
'bakery' → 'Grocery Supermarket': 15 value(s) replaced
'bar & restaurant' → 'HorecaCafe': 2 value(s) replaced
'bar &restaurant' → 'HorecaCafe': 40 value(s) replaced
'bar&restaurant' → 'HorecaCafe': 20 value(s) replaced
'beverage wholesaler' → 'Liquor Wholesale': 39 value(s) replaced
'butcheries' → 'Butchery': 2 value(s) replaced
'butchery' → 'Butchery': 674 value(s) replaced
'butchery (cooler) p(ii)' → 'Butchery': 14 value(s) replaced
'butchery (cooler) p(iii)' → 'Butchery': 11 value(s) replaced
'butchery (non-cooler) p(ii)' → 'Butchery': 5 value(s) replaced
'butchery (non-cooler) p(iii)' → 'Butchery': 13 value(s) replaced
'butchery (non-cooler) p(iv)' → 'Butchery': 6 value(s) replaced
'cafe' → 'HorecaCafe': 22 value(s) replaced
'café' → 'HorecaCafe': 7 value(s) replaced
'cafe &all e&d' → 'HorecaCafe': 19 value(s) replaced
'café &al

### 🧩 Create 'PICOS Sub Channel' and 'Market Segment' Columns

This step generates two key classification columns:

- `PICOS Sub Channel`: Combination of `Trade Channel`, `Coke Cooler Availability`, and `PICOS_Region`.
- `Market Segment`: Combination of `PICOS Sub Channel` and `Leveling`.

Both fields are normalized and stripped of extra spaces for consistency.


In [182]:
# === 🧩 Generate 'PICOS Sub Channel' and 'Market Segment' Columns ===

# Step 1: Clean input columns
dirty_df['Trade Channel'] = dirty_df['Trade Channel'].astype(str).str.strip()
dirty_df['Coke Cooler Availability'] = dirty_df['Coke Cooler Availability'].astype(str).str.strip()
dirty_df['PICOS_Region'] = dirty_df['PICOS_Region'].astype(str).str.strip()
dirty_df['Leveling'] = dirty_df['Leveling'].astype(str).str.strip()

# Step 2: Create 'PICOS Sub Channel' column
dirty_df['PICOS Sub Channel'] = (
    dirty_df['Trade Channel'] + " (" +
    dirty_df['Coke Cooler Availability'] + ") " +
    dirty_df['PICOS_Region']
).str.strip()

# Step 3: Create 'Market Segment' column
dirty_df['Market Segment'] = (
    dirty_df['PICOS Sub Channel'] + " " +
    dirty_df['Leveling']
).str.strip()

# Step 4: Export the updated DataFrame
output_path = "/content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/dirty_df_with_segments.xlsx"
dirty_df.to_excel(output_path, index=False)

# Step 5: Confirmation message
print("✅ Columns 'PICOS Sub Channel' and 'Market Segment' created successfully.")
print(f"📁 File saved to: {output_path}")


✅ Columns 'PICOS Sub Channel' and 'Market Segment' created successfully.
📁 File saved to: /content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/dirty_df_with_segments.xlsx


### 👤 Map Surveyor Numbers to User Names

This step links each record in `dirty_df` to the corresponding **User Name** from `user_md_df` using `SUR NO` ↔ `Surv. No.` as the lookup key.

- Unmatched values are labeled as **"Not Found"**


In [183]:
# === 👤 Map 'Surv. No.' to 'UserName' Using users_df ===

# Step 1: Normalize keys for matching
users_df['SUR NO'] = users_df['SUR NO'].astype(str).str.strip()
dirty_df['Surv. No.'] = dirty_df['Surv. No.'].astype(str).str.strip()

# Step 2: Create lookup dictionary
user_lookup = users_df.set_index('SUR NO')['User Name'].to_dict()

# Step 3: Map values to dirty_df
dirty_df['UserName'] = dirty_df['Surv. No.'].map(user_lookup).fillna('Not Found')

# Step 4: Report summary
print("✅ 'UserName' column created.")
print(f"🔢 Unique usernames mapped: {dirty_df['UserName'].nunique()}")
print(f"⚠️ Unmatched entries: {(dirty_df['UserName'] == 'Not Found').sum()}")


✅ 'UserName' column created.
🔢 Unique usernames mapped: 69
⚠️ Unmatched entries: 0


### 🗓️ Add Start and End Dates from Week Calendar

This section enhances the `dirty_df` by:

- Creating two new columns: **`Start Date`** and **`End Date`** based on the numeric `Week day`.
- Looking up the corresponding dates from the `week_dates_df` using `Week day` as a key.
- Providing a summary of mapping results, including:

  - ✅ Total number of matched records
  - ❌ Total number of unmatched records
  - 🔍 List of unique unmatched `Week day` values (if any)


In [184]:
# === Step 1: Clean and convert 'Week day' columns to integers (nullable Int64) ===
dirty_df['Week day'] = pd.to_numeric(dirty_df['Week day'], errors='coerce').astype('Int64')
week_dates_df['Week day'] = pd.to_numeric(week_dates_df['Week day'], errors='coerce').astype('Int64')

# === Step 2: Prepare lookup dictionaries from week_dates_df ===
start_date_map = week_dates_df.set_index('Week day')['Start Date'].to_dict()
end_date_map = week_dates_df.set_index('Week day')['End Date'].to_dict()

# === Step 3: Map the dates to dirty_df based on Week day ===
dirty_df['Start Date'] = dirty_df['Week day'].map(start_date_map)
dirty_df['End Date'] = dirty_df['Week day'].map(end_date_map)

# === Step 4: Calculate summary of mapping ===
total_rows = len(dirty_df)
matched = dirty_df['Start Date'].notna().sum()
unmatched = dirty_df['Start Date'].isna().sum()
unmatched_values = dirty_df[dirty_df['Start Date'].isna()]['Week day'].unique()

# === Step 5: Display summary ===
print("✅ Start Date and End Date columns created successfully.")
print(f"🔢 Total Records: {total_rows}")
print(f"🎯 Matched Records: {matched}")
print(f"⚠️ Unmatched Records: {unmatched}")
if unmatched > 0:
    print("🚫 Unmatched 'Week day' values:")
    for val in unmatched_values:
        print(f" - {val}")


✅ Start Date and End Date columns created successfully.
🔢 Total Records: 25408
🎯 Matched Records: 25407
⚠️ Unmatched Records: 1
🚫 Unmatched 'Week day' values:
 - 8


### 📦 Finalize & Export Cleaned August Dataset

This step creates a new cleaned DataFrame `eth_clean_aug` based on all applied transformations and saves it to the `/clean` directory for August 2025.


In [185]:
# ✅ Assign cleaned DataFrame to a new variable
eth_clean_aug = dirty_df.copy()

# ✅ Define output path
output_path = '/content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/aug_cleaned_file.xlsx'

# ✅ Save the cleaned DataFrame to Excel
eth_clean_aug.to_excel(output_path, index=False)

# ✅ Confirmation
print(f"✅ Cleaned DataFrame saved as 'eth_clean_aug' and written to:\n{output_path}")


✅ Cleaned DataFrame saved as 'eth_clean_aug' and written to:
/content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/aug_cleaned_file.xlsx


### 🧾 Create Final Cleaned Dataset for Maxerience Upload (August)

This step constructs a new standardized DataFrame from `eth_clean_aug` for Maxerience system upload and exports it to Excel.


In [186]:

# === 🧱 Build Final DataFrame for Maxerience ===
clean_df = pd.DataFrame({
    'Country': 'Ethiopia',
    'SubClientCode': '78',
    'SubClient': 'CCBA ETHIOPIA',
    'CustomerCode': eth_clean_aug['MSD Code'],
    'Barcode': '',
    'CustomerName': eth_clean_aug['Customer Name'],
    'Street': eth_clean_aug['Land Mark'],
    'Address2': '',
    'Address3': '',
    'Address4': '',
    'City': eth_clean_aug['DISTRICT'],
    'State': '',
    'PostalCode': '',
    'RegionCode': eth_clean_aug['Region'],
    'RegionName': eth_clean_aug['Region'],
    'Latitude': '',
    'Longitude': '',
    'TimeZone': '(UTC+03:00) Nairobi',
    'PrimaryPhone': '',
    'MobilePhone': '',
    'Email': '',
    'SalesOrganizationCode': eth_clean_aug['RSM Ref.'],
    'SalesOrganizationName': eth_clean_aug['RSM Name'],
    'SalesOfficeCode': eth_clean_aug['ASM Ref.'],
    'SalesOfficeName': eth_clean_aug['ASM Name'],
    'SalesGroupCode': '',
    'SalesGroupName': '',
    'SalesTerritoryCode': eth_clean_aug['CSR Ref.'],
    'SalesTerritory': eth_clean_aug['CSR Name'],
    'TeleSellingTerritoryCode': '',
    'TeleSellingTerritoryName': '',
    'CreditApproverCode': '',
    'CreditApproverName': '',
    'MerchandizerCode': '',
    'MerchandizerName': '',
    'P1_TerritoryCode': '',
    'P1_TerritoryName': '',
    'P2_TerritoryCode': '',
    'P2_TerritoryName': '',
    'P3_TerritoryCode': '',
    'P3_TerritoryName': '',
    'P4_TerritoryCode': '',
    'P4_TerritoryName': '',
    'P5_TerritoryCode': '',
    'P5_TerritoryName': '',
    'ReserveRouteCode': '',
    'ReserveRouteName': '',
    'TradeChannelCode': eth_clean_aug['Trade Channel'],
    'TradeChannelName': eth_clean_aug['Trade Channel'],
    'SubTradeChannelCode': eth_clean_aug['Market Segment'],
    'SubTradeChannel': eth_clean_aug['Market Segment'],
    'LocalTradeChannelCode': '',
    'LocalTradeChannelName': '',
    'LocalSubTradeChannelCode': '',
    'LocalSubTradeChannelName': '',
    'TradeGroupCode': '',
    'TradeGroupName': '',
    'ChainCode': '',
    'ChainName': '',
    'CustomerTierCode': eth_clean_aug['Leveling'],
    'CustomerTierName': eth_clean_aug['Leveling'],
    'CPLCode': eth_clean_aug['OCCD Code'],
    'CPLName': eth_clean_aug['OCCD NAME'],
    'ClusterCode': '',
    'ClusterName': '',
    'LocalClusterCode': eth_clean_aug['Coke Cooler Availability'],
    'LocalClusterName': eth_clean_aug['Coke Cooler Availability'],
    'MarketSegmentCode': eth_clean_aug['Market Segment'],
    'MarketSegmentName': eth_clean_aug['Market Segment'],
    'SegmentCode': '',
    'SegmentName': '',
    'EnvironmentCode': '',
    'EnvironmentName': '',
    'Assortment1': '',
    'Assortment2': '',
    'Assortment3': '',
    'Assortment4': '',
    'Assortment5': '',
    'ExtraFields': '',
    'IsActive': ''
})

# === 💾 Save Output to Clean Folder ===
output_file_path = '/content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/clean_maxerience_aug.xlsx'
clean_df.to_excel(output_file_path, index=False)

# === ✅ Confirmation ===
print(f"✅ Cleaned file saved to: {output_file_path}")


✅ Cleaned file saved to: /content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/clean_maxerience_aug.xlsx


### 📦 Generate iRED MD File for August 2025

This block creates a new structured dataset based on the cleaned `eth_clean_aug` DataFrame for iRED system upload and exports it to Excel.


In [187]:

# === 🧱 Build iRED MD August DataFrame ===
new_df = pd.DataFrame({
    'Country': 'Ethiopia',
    'Client': 'CCBA Ethiopia',
    'Sub Client Code': '78',
    'Sub Client': 'CCBA Ethiopia',
    'outletcode': eth_clean_aug['MSD Code'],
    'Customer Tier': eth_clean_aug['Leveling'],
    'Barcode': 'NO TAG',
    'Customer name': eth_clean_aug['Customer Name'],
    'Address 1': eth_clean_aug['Sub route Number'],
    'Street': eth_clean_aug['Land Mark'],
    'City': eth_clean_aug['DISTRICT'],
    'Region': eth_clean_aug['Region'],
    'Country code': 'ETH',
    'Geo coordinates Lat.': eth_clean_aug['X _ Coordinat'],
    'Geo coordinates Long.': eth_clean_aug['Y - Coordinat'],
    'Sales OrganizationCode': eth_clean_aug['RSM Ref.'],
    'Sales Organization': eth_clean_aug['RSM Name'],
    'Sales officeCode': eth_clean_aug['ASM Ref.'],
    'Sales Office': eth_clean_aug['ASM Name'],
    'Sales groupCode': eth_clean_aug['Sales Territory'],
    'Sales Group': eth_clean_aug['Sales Territory'],
    'BD territory code': eth_clean_aug['CSR Ref.'],
    'BD territory': eth_clean_aug['CSR Name'],
    'users': eth_clean_aug['UserName'],
    'Trade Channel': eth_clean_aug['Trade Channel'],
    'Sub-Channel': eth_clean_aug['PICOS Sub Channel'],
    'PICOS Channel & Tier': eth_clean_aug['Market Segment'],
    'Date': '2025/08/01',
    'Month': 'Aug',
    'Year': '2025',
    'OCCD No': eth_clean_aug['OCCD Code'],
    'OCCD Name': eth_clean_aug['OCCD NAME'],
    'Cooler Tier': eth_clean_aug['Coke Cooler Availability']
})

# === 💾 Export to Excel ===
new_file_path = "/content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/iRED_MD_aug.xlsx"
new_df.to_excel(new_file_path, index=False)

# === ✅ Confirmation ===
print(f"✅ New iRED MD file saved to: {new_file_path}")


✅ New iRED MD file saved to: /content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/iRED_MD_aug.xlsx


### 🗺️ Generate Route Schedule File

This block creates a simple route schedule from the `eth_clean_aug` dataset with static visit sequencing, and exports it to Excel.

In [188]:

# === 📋 Create Route Schedule DataFrame ===
route_schedule = pd.DataFrame({
    'UserName': eth_clean_aug['UserName'],
    'OutletCode': eth_clean_aug['MSD Code'],
    'Sequence': 1,
    'PlannedVisitId': 0,
    'StartDate': eth_clean_aug['Start Date'],
    'EndDate': eth_clean_aug['End Date']
})

# === 💾 Export to Excel ===
route_schedule_path = "/content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/Route_Schedule.xlsx"
route_schedule.to_excel(route_schedule_path, index=False)

# === ✅ Confirmation ===
print(f"✅ Route schedule saved to: {route_schedule_path}")


✅ Route schedule saved to: /content/drive/MyDrive/SNS/Ethiopia/2025/aug/clean/Route_Schedule.xlsx


In [189]:
# 📌 At the very bottom of the notebook
notebook_end_time = time.time()
elapsed_seconds = notebook_end_time - notebook_start_time

# Convert to HH:MM:SS
elapsed_time_str = time.strftime("%H:%M:%S", time.gmtime(elapsed_seconds))

print(f"✅ Notebook executed in: {elapsed_time_str}")


✅ Notebook executed in: 00:04:55
